# Librerías

Usaremos principalmente BeautifulSoup y selenium para poder ayudarnos con el web scrapping. Para almacenar y mostrar los datos obtenidos, se utilizará Pandas

In [53]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd 


# Preprocesamiento y trabajo previo

Primero, obtendremos todos los datos requeridos buscando por su respectivo tag de class en el html parseado por el BeautifulSoup\
__Nombre Local__: "result-title"\
__Direccion__: "address-text"\
__Calificación__: "prw_rup prw_common_responsive_rating_and_review_count"\
__Numero de opiniones__: (está dentro del class de calificación) "a: review_count"\
__Número de menciones__: "review-mention-block"

In [54]:
# hallamos todos los divs correspondientes con ayuda de bs
def hallarDivs(soup):
    nombresHTML = soup.find_all("div", {"class": "result-title"}) 
    direccionHTML = soup.find_all("div", {"class":  "address-text"}) 
    calificacionHTML = soup.find_all("div", {"class":  "prw_rup prw_common_responsive_rating_and_review_count"})  
    numeroOpinionesHTML = soup.find_all("div", {"class": "prw_rup prw_common_responsive_rating_and_review_count"})
    numeroMencionesHTML = soup.find_all("div", {"class": "review-mention-block"})
    return nombresHTML, direccionHTML, calificacionHTML, numeroOpinionesHTML, numeroMencionesHTML


Lo anterior nos retorna todo el div con el html en donde se encuentra el atributo requerido, por lo que debemos extraer lo que nos importa (que sería el nombre del local, la dirección, la calificación, el número de opiniones y el número de menciones)

In [55]:
def procesamientoHTML(nombresHTML, direccionHTML, calificacionHTML, numeroOpinionesHTML, numeroMencionesHTML):    
    # para nombres
    listaLocales = []
    for nombre in nombresHTML:
        listaLocales.append(nombre.span.text)

    # para direccion
    listaDirecciones = []
    for direccion in direccionHTML:
        listaDirecciones.append(direccion.text)

    # para calificacion
    listaCalificaciones = []
    for calificacion in calificacionHTML:
        tempCal = calificacion.span.get('alt').split()[0] # obtenemos calificacion en string
        cal = tempCal.replace(",",".") # no podemos pasar a float si tiene una coma
        listaCalificaciones.append(float(cal))

    # para numero de opiniones
    listaNumOpiniones = []
    for numOpiniones in numeroOpinionesHTML:
        tempOp = numOpiniones.text.split()[0] # obtenemos numero de opiniones en string
        op = tempOp.replace(".","") # quitamos el . para pasarlo a entero
        listaNumOpiniones.append(int(op))

    # para numero de menciones
    listaNumMenciones = []
    for numMenciones in numeroMencionesHTML:
        tempMen = numMenciones.text.split()[0] # obtenemos numero de menciones en string
        men = tempMen.replace(".","") # quitamos el . para pasarlo a entero
        listaNumMenciones.append(int(men))
    
    # retornamos los 10 primeros locales, ya que así nos lo piden en el caso
    return listaLocales[:10], listaDirecciones[:10], listaCalificaciones[:10], listaNumOpiniones[:10], listaNumMenciones[:10]



# Visualización de Datos con Pandas

Finalmente, hacemos uso de las funciones creadas anteriormente y las librerías importadas para poder visualizar los datos que se obtengan. Para que se realice una búsqueda de forma parametrizada, se realizará una función que nos mostrará por pantalla un dataframe con lo datos obtenidos, la cual recibirá como parámetros la consulta de tipo de comida que se esté buscando y opcionalmente, se puede agregar el parámetro de la localidad que quiero que sea el tipo de comida que estoy buscando, el cual en este caso será un entero que denota un ID de geolocalización. Lamentablemente, estos ID's tuvieron que obtenerse manualmente, haciendo la consulta uno mismo en el navegador y sacando el ID de la url a la que me redirigía al filtrar por una localidad en particular. Como este parámetro es opcional, si no se agrega, simplementa busca en locales de todo el mundo.

In [56]:
def mostrarDatos(queryFood, queryPlace=None):
    # primero parseamos en el driver por comida, para luego unir la url nueva obtenida por el driver con la localizacion
    url = 'https://www.tripadvisor.cl/Search?q=' + queryFood
    driver = webdriver.Chrome()

    driver.get(url)
        
    # obtenemos de nuevo la url que redirige en el navegador
    url = driver.current_url + "&geo=" + str(queryPlace)
    driver.get(url)
    
    # parseamos la pagina a html con bs
    soup = BeautifulSoup(driver.page_source, 'html.parser') 

    # utilizamos las funciones creadas anteriormente para pasarle los datos al dataframe
    nombresHTML, direccionHTML, calificacionHTML, numeroOpinionesHTML, numeroMencionesHTML = hallarDivs(soup)
    listaLocales, listaDirecciones, listaCalificaciones, listaNumOpiniones, listaNumMenciones = procesamientoHTML(nombresHTML, direccionHTML, calificacionHTML, numeroOpinionesHTML, numeroMencionesHTML)

    # diccionario de listas creadas con los datos anteriores 
    dicc = {'Local': listaLocales,
            'Direccion': listaDirecciones, 
            'Calificacion (0 a 5)': listaCalificaciones,
            'Numero de Opiniones': listaNumOpiniones,
            'Numero de Menciones': listaNumMenciones} 

    df = pd.DataFrame(dicc)
    display(df)


In [57]:
# Hamburguesas de santiago
mostrarDatos("hamburguesas", 294305)

,Local,Direccion,Calificacion (0 a 5),Numero de Opiniones,Numero de Menciones
0,La Burguesía,"Santa Magdalena 99, Santiago, Región Metropoli...",4.5,945,651
1,Holy Moly,"Merced 461, Santiago, Región Metropolitana de ...",4.5,438,201
2,Uncle Fletch,"Dardignac 0192, Santiago, Región Metropolitana...",4.5,2569,1415
3,Uncle Fletch Plaza Nunoa,"Jorge Washington 111, Santiago, Región Metropo...",4.5,1002,552
4,Streat Burger,"Isidora Goyenechea 3199, Santiago, Región Metr...",4.0,335,218
5,El Honesto Mike,"Avenida Los Leones 96, Santiago, Región Metrop...",4.0,257,164
6,La Antojeria,"Eleodoro Yanez 1049, Santiago, Región Metropol...",4.0,166,99
7,Mr. Jack,"Av. Pdte. Kennedy Nº5413, Santiago, Región Met...",4.0,559,310
8,Tren Uncle Fletch,"Avenida El Rodeo 12850, Santiago, Región Metro...",4.5,347,134
9,Mr. Jack,"Calle Tabancura 1108, Santiago, Región Metropo...",4.0,386,230


In [58]:
# Sushis de santiago
mostrarDatos("sushi", 294305)

,Local,Direccion,Calificacion (0 a 5),Numero de Opiniones,Numero de Menciones
0,Naoki,"Avenida Vitacura 3875, Santiago, Región Metrop...",4.5,1135,209
1,Panko,"José Victorino Lastarria 53, Santiago, Región ...",4.5,670,205
2,Mikan Sushi,"Avenida Portugal, 471, Santiago, Región Metrop...",4.5,77,50
3,Aomori,"Toesca 1920 B, Santiago, Región Metropolitana ...",4.5,143,72
4,Senz Sushi & Nikkel,"Avenida Andrés Bello 2447, Santiago, Región Me...",3.5,573,209
5,Ichiban,"Avenida Padre Hurtado Norte 1521, Santiago, Re...",4.5,562,167
6,Republica Nikkei,"Merced 571, Santiago, Región Metropolitana de ...",4.5,198,29
7,Tataki Express,"Luis Pateur 6052, Santiago, Región Metropolita...",4.5,59,38
8,Inari Nikkei Sushi,"Llano Subercaseux 3023, Santiago, Región Metro...",4.5,85,48
9,Duri Sushi,"Calle Agustinas, 984, Santiago, Región Metropo...",4.0,154,75


In [59]:
# pizzas de nueva york
mostrarDatos("pizzas", 60763)

,Local,Direccion,Calificacion (0 a 5),Numero de Opiniones,Numero de Menciones
0,Bleecker Street Pizza,"69 7th Ave S, Nueva York, Estado de Nueva York",4.5,1529,44
1,NY Pizza Suprema,"413 8th Ave, Nueva York, Estado de Nueva York",4.5,1286,83
2,Joe's Pizza - Carmine St,"7 Carmine St, Nueva York, Estado de Nueva York",4.5,1586,117
3,John's of Times Square,"260 W 44th St, Nueva York, Estado de Nueva York",4.0,6059,214
4,Joe's Pizza,"1435 Broadway, Nueva York, Estado de Nueva York",4.5,456,33
5,Rubirosa,"235 Mulberry St, Nueva York, Estado de Nueva York",4.5,1273,81
6,John's of Bleecker Street,"278 Bleecker St, Nueva York, Estado de Nueva York",4.5,1796,58
7,Eataly NYC Flatiron,"200 5th Avenue, Nueva York, Estado de Nueva York",4.5,13131,254
8,Don Antonio,"309 W 50th St, Nueva York, Estado de Nueva York",4.5,2175,53
9,Prince Street Pizza,"27 Prince St, Nueva York, Estado de Nueva York",4.5,436,27


In [60]:
# hamburguesas de todo el mundo
mostrarDatos("hamburguesas")

,Local,Direccion,Calificacion (0 a 5),Numero de Opiniones,Numero de Menciones
0,Shake Shack,"1216 18th St NW, Washington DC, Distrito de Co...",4.5,8609,1200
1,Burgermeister Schlesisches Tor,"Oberbaumstrasse 8, Berlín, Alemania",4.5,5139,721
2,Steakburger Gran Vía 16,"Gran Vía, 16, Madrid, España",4.5,3349,775
3,La Bistroteca,"Calle Espartinas 7, local izquierda, Madrid, E...",4.5,763,589
4,Mu. Burgerhouse,"Avenida 40 Nte No. 11, Playa del Carmen, Penín...",5.0,1799,750
5,Burnout,"Calle Valverde 6, Madrid, España",4.5,1198,720
6,Shake Shack,"691 8th Ave, Nueva York, Estado de Nueva York",4.5,6369,656
7,Bacoa,"Marques de L'Argentera, 1 Bis, Barcelona, Cata...",4.5,2675,746
8,Bacoa,"Ronda Universidad 31, Barcelona, Cataluña, España",4.5,1759,664
9,The Burger Joint,"Calle Jorge Luis Borges 1776, Buenos Aires, Di...",4.5,1832,675


In [61]:
# comida rapida de Madrid
mostrarDatos("comida%20rapida", 187514) # notar que, si queremos buscar un tipo de comida
                                        # que tenga espacios en su nombre, debemos separarlo con '%20'

,Local,Direccion,Calificacion (0 a 5),Numero de Opiniones,Numero de Menciones
0,Kathmandu Tandoori House,"Calle Acuario, 11, Madrid, España",5.0,2398,58
1,Steakburger Gran Vía 16,"Gran Vía, 16, Madrid, España",4.5,3349,78
2,Takos Al Pastor,"Calle Abada 2, Madrid, España",4.5,1257,106
3,Tierra Burrito Bar,"Calle Guzman El Bueno 52, Madrid, España",4.0,748,102
4,Five Guys,"Calle Gran Via 44, Madrid, España",3.5,1104,155
5,Zenith Brunch & Cocktails,"Calle Valverde 28, Madrid, España",5.0,1582,35
6,Wok to Walk,"Calle Mayor 4, Madrid, España",4.0,361,42
7,Honest Greens Castellana,"Paseo Castellana 89, Madrid, España",4.0,713,64
8,Wok to Walk,"Calle Hortaleza, 7, Madrid, España",4.0,214,32
9,Taco Bell,"Calle de Preciados, 29, Madrid, España",3.5,134,59
